![Transfer Learning](./transfer_learning.jpg "Transfer Learning")

![VGG16](./vgg16.webp "VGG16")

![VGG16](./vgg16_2.webp "VGG16")

In [1]:
!git clone https://github.com/WaShindeiru/pytorch-pruning

Cloning into 'pytorch-pruning'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 62 (delta 25), reused 47 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (62/62), 234.03 KiB | 1.24 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip drive/MyDrive/animals10_small.zip

Streaming output truncated to the last 5000 lines.
  inflating: animals10_small/train/cow/OIP-hxOgPSYXFirZiYYj22LVPgHaFm.jpeg  
  inflating: animals10_small/train/cow/OIP-nu0nzd4-vVlOnbnXdMDCcQHaFc.jpeg  
  inflating: animals10_small/train/cow/OIP-emebs7yJv0R0AEqNyd4n3wHaFj.jpeg  
  inflating: animals10_small/train/cow/OIP-ly01_XT8mCNBXvOR6CywKwHaFZ.jpeg  
  inflating: animals10_small/train/cow/OIP-f_Vaso8HY7syrWULazARswHaE7.jpeg  
  inflating: animals10_small/train/cow/OIP-kWvMHMSaFR9LBHMXnqndtQHaFT.jpeg  
  inflating: animals10_small/train/cow/OIP-fBSWAhQajcG7tqDbKs0JnQHaFF.jpeg  
  inflating: animals10_small/train/cow/OIP-hcLyUUJi_4ABa8fBCANF9AHaFQ.jpeg  
  inflating: animals10_small/train/cow/OIP-YT9-aM3XtMHBSRLI4fyX7gHaFc.jpeg  
  inflating: animals10_small/train/cow/OIP-jWrRPqSCcv1o_SqI11LYUAHaFj.jpeg  
  inflating: animals10_small/train/cow/OIP-qGuyYzG52QpEgd2KrUZDswHaEy.jpeg  
  inflating: animals10_small/train/cow/OIP-nRJIWm0m9DqrCOhtpwpeowHaFj.jpeg  
  inflating: animals10_sm

In [13]:
!cp pytorch-pruning/dataset.py dataset.py
!cp pytorch-pruning/finetune.py finetune.py
!cp pytorch-pruning/prune.py prune.py

In [14]:
import torch
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
from time import time

import  dataset
from finetune import ModifiedVGG16Model

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
def evaluate_model(model_, test_loader_, device_, threshold=None):
    model_.eval()
    all_preds = []
    all_labels = []
    batch_times = []
    total_inference_time = 0.
    i = 0

    with torch.no_grad():
        for batch, labels in test_loader_:
            if threshold is not None:
                if i >= threshold:
                    break

            batch, labels = batch.to(device_), labels.to(device_)

            batch_start = time()
            outputs = model_(batch)
            batch_time = time() - batch_start
            batch_times.append(batch_time)
            total_inference_time += batch_time

            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            if threshold is not None:
                i += batch.shape[0]

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    accuracy = (all_preds == all_labels).mean()
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='macro', zero_division=0
    )

    avg_batch_time = np.mean(batch_times) * 1000  # ms

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "avg_batch_time": avg_batch_time,
    }

In [18]:
def print_result(cache):
  accuracy = cache["accuracy"]
  print(f"Accuracy: {np.mean(accuracy):.4f}")
  precision = cache["precision"]
  print(f"Precision: {np.mean(precision):.4f}")
  recall = cache["recall"]
  print(f"Recall: {np.mean(recall):.4f}")
  f1 = cache["f1"]
  print(f"F1-Score: {np.mean(f1):.4f}")
  avg_time = cache["avg_batch_time"]
  print(f"Average batch time: {np.mean(avg_time):.4f} ms")

In [19]:
def compute_mean(model_, test_loader_, device_, num_iter):
    cache = {
        "accuracy": [],
        "precision": [],
        "recall": [],
        "f1": [],
        "avg_batch_time": []
    }

    for _ in range(num_iter):
        temp = evaluate_model(model_, test_loader_, device_)
        cache["accuracy"].append(temp["accuracy"])
        cache["precision"].append(temp["precision"])
        cache["recall"].append(temp["recall"])
        cache["f1"].append(temp["f1"])
        cache["avg_batch_time"].append(temp["avg_batch_time"])

    # print(cache["avg_batch_time"])
    # print(sum(cache["avg_batch_time"]))
    # print(sum(cache["avg_batch_time"]) / len(cache["avg_batch_time"]))

    print_result(cache)

### VGG16 GPU

In [22]:
!cp drive/MyDrive/model_vg16_09_May_22:18.pth model_vg16_09_May_22:18.pth
!cp drive/MyDrive/model_vg16_prunned_10_May_14:55 model_vg16_prunned_10_May_14:55.pth

In [20]:
test_path = "./animals10_small/test"

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_loader = dataset.test_loader(test_path)
model = ModifiedVGG16Model()
model.load_state_dict(torch.load("model_vg16_09_May_22:18.pth", map_location=device))
model = model.to(device)

compute_mean(model, test_loader, device, 4)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: T

Accuracy: 0.9521
Precision: 0.9531
Recall: 0.9521
F1-Score: 0.9522
Average batch time: 9.4083 ms


### VGG16 pruned GPU

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_loader = dataset.test_loader(test_path)
model = torch.load("model_vg16_prunned_10_May_14:55.pth", map_location=device, weights_only=False)
model = model.to(device)

compute_mean(model, test_loader, device, 4)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: T

Accuracy: 0.9343
Precision: 0.9372
Recall: 0.9343
F1-Score: 0.9341
Average batch time: 6.6853 ms


### VGG16 CPU

In [26]:
device = torch.device("cpu")
test_loader = dataset.test_loader(test_path)
model = ModifiedVGG16Model()
model.load_state_dict(torch.load("model_vg16_09_May_22:18.pth", map_location=device))
model = model.to(device)
result = evaluate_model(model, test_loader, device, 100)
print_result(result)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Accuracy: 0.9375
Precision: 0.9398
Recall: 0.9381
F1-Score: 0.9353
Average batch time: 13336.0476 ms


### VGG16 pruned CPU

In [28]:
device = torch.device("cpu")
test_loader = dataset.test_loader(test_path)
model = torch.load("model_vg16_prunned_10_May_14:55.pth", map_location=device, weights_only=False)
result = evaluate_model(model, test_loader, device, 100)
print_result(result)

Accuracy: 0.9609
Precision: 0.9630
Recall: 0.9657
F1-Score: 0.9631
Average batch time: 7035.5358 ms


## Liczba parametrów

VGG16

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_loader = dataset.test_loader(test_path)
model = ModifiedVGG16Model()
model.load_state_dict(torch.load("model_vg16_09_May_22:18.pth", map_location=device))
model = model.to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"VG16 łączna liczba parameterów: {total_params}")

total_conv_params = sum(p.numel() for p in model.features.parameters())
print(f"VG16 część konwolucyjna, liczba parameterów: {total_conv_params}")

total_class_params = sum(p.numel() for p in model.classifier.parameters())
print(f"VG16 część klasyfikatora, liczba parameterów: {total_class_params}")

VG16 łączna liczba parameterów: 134301514
VG16 część konwolucyjna, liczba parameterów: 14714688
VG16 część klasyfikatora, liczba parameterów: 119586826


VGG16 pruned

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_loader = dataset.test_loader(test_path)
model = torch.load("model_vg16_prunned_10_May_14:55.pth", map_location=device, weights_only=False)
model = model.to(device)

total_params_pruned = sum(p.numel() for p in model.parameters())
print(f"VG16 łączna liczba parameterów: {total_params_pruned}")

total_conv_params_pruned = sum(p.numel() for p in model.features.parameters())
print(f"VG16 część konwolucyjna, liczba parameterów: {total_conv_params_pruned}")

total_class_params_pruned = sum(p.numel() for p in model.classifier.parameters())
print(f"VG16 część klasyfikatora, liczba parameterów: {total_class_params_pruned}")

VG16 łączna liczba parameterów: 38664702
VG16 część konwolucyjna, liczba parameterów: 2169332
VG16 część klasyfikatora, liczba parameterów: 36495370


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


| Model         | Łączna liczba parametrów     | Część konwolucyjna, liczba parametrów     | Część klasyfikatora, liczba parameterów     | Rozmiar pliku |
|---------------|------------------------------|-------------------------------------------|---------------------------------------------|---------------|
| VGG 16        | 134 301 514                  | 14 714 688                                | 119 586 826                                 | 537.2MB       |
| VGG 16 pruned | 38 664 702                   | 2 169 332                                 | 36 495 370                                  | 154.7MB       |

### Liczba usuniętych filtrów

In [ ]:
iter_1 = {21: 57, 17: 61, 28: 157, 26: 68, 14: 16, 19: 49, 12: 12, 24: 56, 5: 8, 10: 20, 7: 3, 2: 2, 0: 3}
iter_2 = {28: 113, 12: 23, 21: 61, 26: 96, 17: 48, 19: 50, 24: 73, 0: 4, 14: 13, 10: 15, 7: 12, 5: 3, 2: 1}
iter_3 = {17: 53, 24: 77, 12: 20, 21: 81, 19: 64, 26: 79, 28: 55, 14: 31, 10: 22, 5: 11, 7: 7, 2: 6, 0: 6}
iter_4 = {7: 10, 17: 77, 19: 65, 10: 29, 2: 6, 28: 47, 26: 65, 21: 69, 24: 72, 12: 27, 14: 24, 0: 5, 5: 16}
iter_5 = {7: 21, 14: 39, 24: 42, 2: 6, 19: 80, 17: 66, 12: 39, 5: 14, 21: 56, 26: 68, 0: 4, 10: 35, 28: 42}

removed_filters = {}

for temp in (iter_1, iter_2, iter_3, iter_4, iter_5):
    for k,v in temp.items():
        if k in removed_filters:
            removed_filters[k] += v
        else:
            removed_filters[k] = v

keys = sorted(removed_filters.keys())
for key in keys:
    print(f"{key}: {removed_filters[key]}")

| **Numer warstwy**             | 0  | 2  | 5  | 7  | 10  | 12  | 14  | 17  | 19  | 21  | 24  | 26  | 28  |
|-------------------------------|----|----|----|----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
| **Liczba usuniętych filtrów** | 22 | 21 | 52 | 53 | 121 | 121 | 123 | 305 | 308 | 324 | 320 | 376 | 414 |